In [9]:
# Install necessary libs
!pip install pandas
import pandas as pd
import sqlite3
import os


In [10]:
folders = ['raw', 'processed', 'output']

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created:", folders)


Folders created: ['raw', 'processed', 'output']


In [15]:
import pandas as pd

raw_path = "/content/retail_sales_dataset.csv"

df_raw = pd.read_csv(raw_path)

# Save a copy into raw/
df_raw.to_csv("raw/retail_sales_raw.csv", index=False)

print("Raw data loaded")
print(df_raw.shape)
df_raw.head()


Raw data loaded
(1000, 11)


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Region,Total Revenue
0,1,24-11-2023,CUST001,Male,34,Beauty,3,50,150,East,0.06
1,2,27-02-2023,CUST002,Female,26,Clothing,2,500,1000,Oceania,0.00
2,3,13-01-2023,CUST003,Male,50,Electronics,1,30,30,Oceania,0.03
3,4,21-05-2023,CUST004,Male,37,Clothing,1,500,500,Central,0.00
4,5,06-05-2023,CUST005,Male,30,Beauty,2,50,100,Africa,0.04


In [12]:
source_path = "/content/retail_sales_dataset.csv"
raw_path = "raw/retail_sales_raw.csv"

# Copy file into raw folder
df_raw = pd.read_csv(source_path)
df_raw.to_csv(raw_path, index=False)

print("Raw data saved to:", raw_path)
print(df_raw.head())


Raw data saved to: raw/retail_sales_raw.csv
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  24-11-2023     CUST001    Male   34           Beauty   
1               2  27-02-2023     CUST002  Female   26         Clothing   
2               3  13-01-2023     CUST003    Male   50      Electronics   
3               4  21-05-2023     CUST004    Male   37         Clothing   
4               5  06-05-2023     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount   Region  Total Revenue  
0         3              50           150     East           0.06  
1         2             500          1000  Oceania           0.00  
2         1              30            30  Oceania           0.03  
3         1             500           500  Central           0.00  
4         2              50           100   Africa           0.04  


In [17]:
df = df_raw.copy()

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Remove duplicates
df.drop_duplicates(inplace=True)

# Handle missing values
df['quantity'] = df['quantity'].fillna(0)
df['price_per_unit'] = df['price_per_unit'].fillna(0)

# If Sales/Revenue exists (assuming it would be 'total_revenue' or 'sales' after standardization)
# Check for 'total_revenue' as 'Sales' is not in original df and 'sales' is usually calculated
if 'total_revenue' in df.columns:
    df['total_revenue'] = df['total_revenue'].fillna(df['quantity'] * df['price_per_unit'])

print("After cleaning:", df.shape)


After cleaning: (1000, 11)


In [20]:
# Standardize column names (already done in previous cell, but kept for self-containment)
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# Convert datatypes
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')

# Create 'sales' column if it doesn't exist, using quantity * price_per_unit
# This addresses the KeyError if 'sales' is not in the original dataframe.
if 'sales' not in df.columns:
    df['sales'] = df['quantity'] * df['price_per_unit']

df['sales'] = pd.to_numeric(df['sales'], errors='coerce')

df.dtypes

,0
transaction_id,int64
date,datetime64[ns]
customer_id,object
gender,object
age,int64
product_category,object
quantity,int64
price_per_unit,int64
total_amount,int64
region,object


In [22]:
# Cost assumption (70% of price)
df['cost'] = df['price_per_unit'] * 0.7

# Profit & Margin
df['profit'] = df['sales'] - (df['quantity'] * df['cost'])
df['margin'] = df['profit'] / df['sales']

# Segment flags
df['high_value_sale'] = df['sales'] > 1000
df['bulk_order'] = df['quantity'] >= 10

df[['sales', 'profit', 'margin', 'high_value_sale', 'bulk_order']].head()


,sales,profit,margin,high_value_sale,bulk_order
0,150,45.0,0.3,False,False
1,1000,300.0,0.3,False,False
2,30,9.0,0.3,False,False
3,500,150.0,0.3,False,False
4,100,30.0,0.3,False,False


In [14]:
df = df_raw.copy()

# 1. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Convert date column
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 3. Convert numeric columns
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')

# 4. Create sales/revenue if missing
if 'sales' not in df.columns:
    df['sales'] = df['quantity'] * df['price_per_unit']

# 5. Handle missing values
df['quantity'].fillna(0, inplace=True)
df['price_per_unit'].fillna(0, inplace=True)
df['sales'].fillna(0, inplace=True)

# 6. Remove invalid dates
df.dropna(subset=['date'], inplace=True)

print("Transformed data preview:")
df.head()


Transformed data preview:


/tmp/ipython-input-1771923145.py:7: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')
/tmp/ipython-input-1771923145.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['quantity'].fillna(0, inplace=True)
/tmp/ipython-input-1771923145.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inp

,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,region,total_revenue,sales
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,East,0.06,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,Oceania,0.00,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,Oceania,0.03,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,Central,0.00,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,Africa,0.04,100


In [23]:
df.to_csv("processed/retail_sales_processed.csv", index=False)
print("Processed data saved")


Processed data saved


In [25]:
#Customers Table

customers = df[['region']].drop_duplicates().reset_index(drop=True)
customers['customer_id'] = customers.index + 1

customers.to_csv("output/customers.csv", index=False)

In [27]:
products = df[['product_category', 'price_per_unit']].drop_duplicates().reset_index(drop=True)
products['product_id'] = products.index + 1

products.to_csv("output/products.csv", index=False)


In [29]:
orders = df[['date', 'region', 'product_category',
             'quantity', 'price_per_unit', 'sales', 'profit', 'margin']]

orders.to_csv("output/orders.csv", index=False)


In [30]:
!ls


output	processed  raw	retail_sales_dataset.csv  sample_data


In [31]:
!ls raw
!ls processed
!ls output


retail_sales_raw.csv
retail_sales_processed.csv
customers.csv  orders.csv  products.csv


In [32]:
import pandas as pd

customers = pd.read_csv("output/customers.csv")
products = pd.read_csv("output/products.csv")
orders = pd.read_csv("output/orders.csv")

customers.to_csv("output/customers.csv", index=False)
products.to_csv("output/products.csv", index=False)
orders.to_csv("output/orders.csv", index=False)

print("CSV files exported successfully")


CSV files exported successfully


In [39]:
!ls
!ls output

output	processed  raw	retail_sales_dataset.csv  sample_data
customers.csv  orders.csv  products.csv  retail_sales.db


In [33]:
import sqlite3

conn = sqlite3.connect("output/retail_sales.db")

In [34]:
customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)

conn.commit()
print("Data loaded into SQLite successfully")


Data loaded into SQLite successfully


In [35]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql(query, conn)


,name
0,customers
1,products
2,orders


In [36]:
pd.read_sql("SELECT * FROM orders LIMIT 5;", conn)


,date,region,product_category,quantity,price_per_unit,sales,profit,margin
0,2023-11-24,East,Beauty,3,50,150,45.0,0.3
1,2023-02-27,Oceania,Clothing,2,500,1000,300.0,0.3
2,2023-01-13,Oceania,Electronics,1,30,30,9.0,0.3
3,2023-05-21,Central,Clothing,1,500,500,150.0,0.3
4,2023-05-06,Africa,Beauty,2,50,100,30.0,0.3


In [47]:
conn.close()
print("Connection closed")

Connection closed


In [48]:
pd.read_sql("SELECT * FROM orders LIMIT 5;", conn)


ProgrammingError: Cannot operate on a closed database.

In [43]:
try:
    pd.read_sql("SELECT * FROM orders LIMIT 1;", conn)
    print("Connection is still open")
except Exception as e:
    print("Connection is closed")
    print("Error message:", e)


Connection is closed
Error message: Cannot operate on a closed database.


In [46]:
import sqlite3

conn = sqlite3.connect("output/retail_sales.db")
print("Connection reopened successfully")

pd.read_sql("SELECT * FROM orders LIMIT 5;", conn)


Connection reopened successfully


,date,region,product_category,quantity,price_per_unit,sales,profit,margin
0,2023-11-24,East,Beauty,3,50,150,45.0,0.3
1,2023-02-27,Oceania,Clothing,2,500,1000,300.0,0.3
2,2023-01-13,Oceania,Electronics,1,30,30,9.0,0.3
3,2023-05-21,Central,Clothing,1,500,500,150.0,0.3
4,2023-05-06,Africa,Beauty,2,50,100,30.0,0.3


In [49]:
import pandas as pd

raw_df = pd.read_csv("raw/retail_sales_raw.csv")
processed_df = pd.read_csv("processed/retail_sales_processed.csv")
orders_df = pd.read_csv("output/orders.csv")


In [50]:
print("Row Count Validation")
print("-" * 40)
print("Raw dataset rows      :", raw_df.shape[0])
print("Processed dataset rows:", processed_df.shape[0])
print("Orders table rows     :", orders_df.shape[0])


Row Count Validation
----------------------------------------
Raw dataset rows      : 1000
Processed dataset rows: 1000
Orders table rows     : 1000


In [51]:
raw_duplicates = raw_df.duplicated().sum()
processed_duplicates = processed_df.duplicated().sum()

print("Duplicate Check")
print("-" * 40)
print("Duplicates in raw data      :", raw_duplicates)
print("Duplicates after processing :", processed_duplicates)


Duplicate Check
----------------------------------------
Duplicates in raw data      : 0
Duplicates after processing : 0


In [52]:
print("Missing Values After Processing")
print("-" * 40)
print(processed_df.isnull().sum())


Missing Values After Processing
----------------------------------------
transaction_id      0
date                0
customer_id         0
gender              0
age                 0
product_category    0
quantity            0
price_per_unit      0
total_amount        0
region              0
total_revenue       0
sales               0
cost                0
profit              0
margin              0
high_value_sale     0
bulk_order          0
dtype: int64


In [53]:
customers_df = pd.read_csv("output/customers.csv")
products_df = pd.read_csv("output/products.csv")

print("Output Table Counts")
print("-" * 40)
print("Customers count:", customers_df.shape[0])
print("Products count :", products_df.shape[0])
print("Orders count   :", orders_df.shape[0])


Output Table Counts
----------------------------------------
Customers count: 13
Products count : 15
Orders count   : 1000


In [54]:
validation_summary = pd.DataFrame({
    "Stage": ["Raw", "Processed", "Orders"],
    "Row_Count": [
        raw_df.shape[0],
        processed_df.shape[0],
        orders_df.shape[0]
    ]
})

validation_summary


,Stage,Row_Count
0,Raw,1000
1,Processed,1000
2,Orders,1000


In [55]:
import pandas as pd
import sqlite3

processed_df = pd.read_csv("processed/retail_sales_processed.csv")

In [56]:
conn = sqlite3.connect("database.sqlite")

In [57]:
processed_df.to_sql(
    name="processed_sales",
    con=conn,
    if_exists="replace",
    index=False
)

conn.commit()
print("processed_data loaded into database.sqlite")

processed_data loaded into database.sqlite


In [58]:
pd.read_sql("SELECT COUNT(*) AS row_count FROM processed_sales", conn)

,row_count
0,1000


In [59]:
from google.colab import files

files.download("processed/retail_sales_processed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
files.download("output/retail_sales.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
!zip -r etl_project_outputs.zip processed output

from google.colab import files
files.download("etl_project_outputs.zip")



updating: processed/ (stored 0%)
updating: processed/retail_sales_processed.csv (deflated 84%)
updating: output/ (stored 0%)
updating: output/customers.csv (deflated 21%)
updating: output/products.csv (deflated 50%)
updating: output/retail_sales.db (deflated 81%)
updating: output/orders.csv (deflated 87%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>